# Introducción a Pandas

En esta sección del taller aprenderemos a utilizar Pandas para el análisis de datos.

Pandas puede considerarse como una versión extremadamente potente de Excel, con muchas más funciones. En concreto, veremos las siguientes funcionalidades de pandas

* Series
* DataFrames
* Datos faltantes
* GroupBy
* Fusión, unión y concatenación
* Operaciones
* Entrada y salida de datos

# Series

El primer tipo de datos principal que aprenderemos sobre pandas es el tipo de datos Series. Importemos Pandas y exploremos el objeto Series.

Una serie es muy similar a una matriz NumPy (de hecho, se basa en el objeto de matriz `numpy`).

Lo que diferencia a la matriz NumPy de una serie es que una serie puede tener etiquetas de eje, lo que significa que se puede indexar por una etiqueta, en lugar de solo por una ubicación numérica. Tampoco necesita contener datos numéricos, sino que puede contener cualquier objeto Python arbitrario.

Exploremos este concepto a través de algunos ejemplos:

In [ ]:
import numpy as np
import pandas as pd

### Creación de una serie

Puede convertir una lista, una matriz numpy o un diccionario en una serie:

In [ ]:
labels = ['a','b','c']
my_list = [10,20,30]
arr = np.array([10,20,30])
d = {'a':10,'b':20,'c':30}

In [ ]:
pd.Series(data=my_list)

In [ ]:
pd.Series(data=my_list,index=labels)

In [ ]:
pd.Series(my_list,labels)

`numpy`arrays

In [ ]:
pd.Series(arr)

In [ ]:
pd.Series(arr,labels)

Diccionarios

In [ ]:
pd.Series(d)

### Datos en una serie

Una serie de pandas puede contener una variedad de tipos de objetos:

In [ ]:
pd.Series(data=labels)

In [ ]:
# Incluso funciones pares
pd.Series([sum,print,len])

## Uso de un índice

La clave para utilizar una serie es comprender su índice. Pandas utiliza estos nombres o números de índice para permitir búsquedas rápidas de información (funciona como una tabla hash o un diccionario).

Veamos algunos ejemplos de cómo obtener información de una serie. Creemos dos series, `ser1` y `ser2`:

In [ ]:
ser1 = pd.Series([1,2,3,4],index = ['USA', 'Germany','USSR', 'Japan'])
ser1

In [ ]:
ser2 = pd.Series([1,2,5,4],index = ['USA', 'Germany','Italy', 'Japan'])
ser2

In [ ]:
ser1['USA']

Las operaciones también se realizan en función del índice:

In [ ]:
ser1 + ser2

# DataFrames

Los DataFrames son el motor de pandas y están directamente inspirados en el lenguaje de programación R. Podemos pensar en un `DataFrame` como un conjunto de objetos Series reunidos para compartir el mismo índice. ¡Usemos pandas para explorar este tema!

In [ ]:
from numpy.random import randn
np.random.seed(101)

df = pd.DataFrame(randn(5,4),index='A B C D E'.split(),columns='W X Y Z'.split())
df

## Selección e indexación

Aprendamos los distintos métodos para extraer datos de un DataFrame.

In [ ]:
df['W']

In [ ]:
df[['W','Z']]

In [ ]:
df.W

Las columnas de DataFrame son simplemente `series`.

In [ ]:
type(df['W'])

Creemos una nueva columna en nuestro dataframe

In [ ]:
df['new'] = df['W'] + df['Y']
df

Borremosla

In [ ]:
df.drop('new',axis=1)
df

In [ ]:
df.drop('new',axis=1,inplace=True)
df

Podemos borrar también filas

In [ ]:
df.drop('E',axis=0)

Seleccionar filas

In [ ]:
df.loc['A']

O seleccionar en función de la posición en lugar de la etiqueta.

In [ ]:
df.iloc[2]

Selección de un subconjunto de filas y columnas

In [ ]:
df.loc['B','Y']

In [ ]:
df.loc[['A','B'],['W','Y']]

### Selección condicional

Una característica importante de pandas es la selección condicional mediante notación entre corchetes, muy similar a numpy:

In [ ]:
df>0

In [ ]:
df[df>0]

In [ ]:
df[df['W']>0]

In [ ]:
df[df['W']>0]['Y']

In [ ]:
df[df['W']>0][['Y','X']]

Para dos condiciones, puede utilizar | y & con paréntesis:

In [ ]:
df[(df['W']>0) & (df['Y'] > 1)]

### Más detalles sobre los índices

Veamos algunas características más de la indexación, como restablecer el índice o configurarlo de otra manera. ¡También hablaremos de la jerarquía de los índices!

In [ ]:
# Reset to default 0,1...n index
df.reset_index()

In [ ]:
newind = 'CA NY WY OR CO'.split()

In [ ]:
df['States'] = newind
df

In [ ]:
df.set_index('States')

In [ ]:
df.set_index('States',inplace=True)
df

## Multiíndice y jerarquía de índices

Repasemos cómo trabajar con el multiíndice. Primero, crearemos un ejemplo rápido de cómo sería un DataFrame con multiíndice:

In [ ]:
outside = ['G1','G1','G1','G2','G2','G2']
inside = [1,2,3,1,2,3]
hier_index = list(zip(outside,inside))
hier_index = pd.MultiIndex.from_tuples(hier_index)
hier_index

In [ ]:
df = pd.DataFrame(np.random.randn(6,2),index=hier_index,columns=['A','B'])
df

Ahora veamos cómo indexar esto. Para la jerarquía del índice utilizamos df.loc[], si esto estuviera en el eje de columnas, solo tendrías que utilizar la notación normal entre corchetes df[]. Al llamar a un nivel del índice se devuelve el sub-dataframe:

In [ ]:
df.loc['G1']

In [ ]:
df.loc['G1'].loc[1]

In [ ]:
df.index.names

In [ ]:
df.index.names = ['Group','Num']
df

In [ ]:
df.xs('G1')

In [ ]:
df.xs(('G1',1))

In [ ]:
df.xs(1,level='Num')

# Datos faltantes

Veamos algunos métodos prácticos para tratar los datos faltantes en pandas:

In [ ]:
df = pd.DataFrame({'A':[1,2,np.nan],
                  'B':[5,np.nan,np.nan],
                  'C':[1,2,3]})
df

In [ ]:
df.dropna()

In [ ]:
df.dropna(axis=1)

In [ ]:
df.dropna(thresh=2)

In [ ]:
df.fillna(value='FILL VALUE')

In [ ]:
df['A'].fillna(value=df['A'].mean())

# `GroupBy`

El método `groupby` permite agrupar filas de datos y llamar a funciones de agregación.

In [ ]:
data = {'Company':['GOOG','GOOG','MSFT','MSFT','FB','FB'],
       'Person':['Sam','Charlie','Amy','Vanessa','Carl','Sarah'],
       'Sales':[200,120,340,124,243,350]}
df = pd.DataFrame(data)
df

 Ahora podemos utilizar el método `groupby` para agrupar filas en función del nombre de una columna. Por ejemplo, agrupemos en función de `Company`. Esto creará un objeto `DataFrameGroupBy`.

In [ ]:
df.groupby('Company')

In [ ]:
by_comp = df.groupby("Company")
by_comp.head()

Ahora podemos llamar a varios métodos de agregacion sobre el objeto `DataFrameGroupBy` creado.

In [ ]:
by_comp.mean(numeric_only=True)

In [ ]:
by_comp.std(numeric_only=True)

In [ ]:
by_comp.min(numeric_only=True)

In [ ]:
by_comp.max(numeric_only=True)

In [ ]:
by_comp.count()

In [ ]:
by_comp.describe()

In [ ]:
by_comp.describe().transpose()

In [ ]:
by_comp.describe().transpose()['GOOG']

# Operaciones

Hay muchas operaciones con `pandas` que nos resultarán muy útiles, pero que no entran en ninguna categoría concreta. Veámos algunas de ellas

In [ ]:
df = pd.DataFrame({'col1':[1,2,3,4],'col2':[444,555,666,444],'col3':['abc','def','ghi','xyz']})
df.head()

In [ ]:
df['col2'].unique()

In [ ]:
df['col2'].nunique()

In [ ]:
df['col2'].value_counts()

Selección de datos

In [ ]:
newdf = df[(df['col1']>2) & (df['col2']==444)]
newdf

Aplicar funciones

In [ ]:
def times2(x):
    return x*2

df['col1'].apply(times2)

In [ ]:
df['col3'].apply(len)

In [ ]:
df['col1'].sum()

¿Cómo podemos borrar de forma permanente una columna?

In [ ]:
del df['col1']
df

También podemos obtener los nombres de las columnas e índices de un `dataframe`

In [ ]:
df.columns

In [ ]:
df.index

O ordenar sus valores

In [ ]:
df.sort_values(by='col2') #inplace=False por defecto

Operaciones relacionadas con los valores nulos

In [ ]:
df.isnull()

In [ ]:
df.dropna()

Podemos incluso rellenar los valores `nan`que tengamos en un `dataframe`

In [ ]:
import numpy as np

df = pd.DataFrame({'col1':[1,2,3,np.nan],
                   'col2':[np.nan,555,666,444],
                   'col3':['abc','def','ghi','xyz']})
df.head()

In [ ]:
df.fillna('FILL')

In [ ]:
data = {'A':['foo','foo','foo','bar','bar','bar'],
     'B':['one','one','two','two','one','one'],
       'C':['x','y','x','y','x','y'],
       'D':[1,3,2,5,4,1]}

df = pd.DataFrame(data)
df

In [ ]:
df.pivot_table(values='D',index=['A', 'B'],columns=['C'])

# Entrada y salida de datos

Pandas puede leer una gran variedad de tipos de archivos utilizando sus métodos `pd.read_`. Echemos un vistazo a los tipos de datos más comunes:

### CSV

In [ ]:
df = pd.read_csv('https://raw.githubusercontent.com/fterroso/curso_ia_smart_cities/refs/heads/main/datos/ejemplo.csv')
df

In [ ]:
df.to_csv('ejemplo2.csv',index=False)

## Excel
Pandas puede leer y escribir archivos Excel, pero ten en cuenta que solo importa datos. No importa fórmulas ni imágenes, ya que la presencia de imágenes o macros puede provocar que el método `read_excel` falle.

In [ ]:
pd.read_excel('https://raw.githubusercontent.com/fterroso/curso_ia_smart_cities/refs/heads/main/datos/Ejemplo_Excel.xlsx')

In [ ]:
df.to_excel('Ejemplo_Excel2.xlsx')

## HTML

Para poder leer un fichero HTML Es posible que tengas que instalar las librearías `htmllib5`, `lxml` y `BeautifulSoup4`.

In [ ]:
!pip install lxml html5lib BeautifulSoup4

In [ ]:
df = pd.read_html('http://www.fdic.gov/bank/individual/failed/banklist.html')

In [ ]:
df[0]

# Caso de uso - Contaminación atmosférica

Vamos ahora a emplear todo lo aprendido para hacer un pequeño análisis inicial de un conjunto de datos que almacena la información sobre partículas contaminantes y niveles de tráfico en diferentes barrios de una ciudad ficticia.

In [ ]:
import pandas as pd

# 1. Leer el CSV
df = pd.read_csv("https://raw.githubusercontent.com/fterroso/curso_ia_smart_cities/refs/heads/main/datos/contaminacion_ejemplo.csv")

In [ ]:
# 2. Mostrar los primeros registros
print("Datos iniciales:")
print(df.head())

In [ ]:
# 3. Calcular medias de contaminantes por barrio
medias = df.groupby("barrio")[["no2", "pm10", "o3"]].mean()
print("\nMedia de contaminantes por barrio:")
print(medias)

In [ ]:
# 4. Detectar el barrio con mayor tráfico promedio
trafico_prom = df.groupby("barrio")["trafico"].mean().sort_values(ascending=False)
print("\nBarrio con más tráfico promedio:")
print(trafico_prom.head(1))

In [ ]:

# 5. Correlación entre tráfico y NO2
corr = df["trafico"].corr(df["no2"])
print(f"\nCorrelación entre tráfico y NO₂: {corr:.2f}")